In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import tensorflow as tf

tf.logging.set_verbosity(tf.logging.INFO)



In [2]:

def cnn_model_fn(features, labels, mode):
  """Model function for CNN."""
  # Input Layer
  # Reshape X to 4-D tensor: [batch_size, width, height, channels]
  # MNIST images are 28x28 pixels, and have one color channel
  input_layer = tf.reshape(features["x"], [-1, 28, 28, 1])

  # Convolutional Layer #1
  # Computes 32 features using a 3x3 filter with ReLU activation.
  # Padding is added to preserve width and height.
  # Input Tensor Shape: [batch_size, 28, 28, 1]
  # Output Tensor Shape: [batch_size, 28, 28, 32]
  conv1 = tf.layers.conv2d(
      inputs=input_layer,
      filters=32,
      kernel_size=[3, 3],
      padding="same",
      activation=tf.nn.relu)


  # Convolutional Layer #2
  # Computes 64 features using a 3x3 filter.
  # Padding is added to preserve width and height.
  # Input Tensor Shape: [batch_size, 28, 28, 32]
  # Output Tensor Shape: [batch_size, 28, 28, 64]
  conv2 = tf.layers.conv2d(
      inputs=conv1,
      filters=64,
      kernel_size=[3, 3],
      padding="same",
      activation=tf.nn.relu)

  # Pooling Layer #1
  # First max pooling layer with a 2x2 filter and stride of 2
  # Input Tensor Shape: [batch_size, 28, 28, 64]
  # Output Tensor Shape: [batch_size, 14, 14, 64]
  pool = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)


  # Add dropout operation; 0.25 probability that element will be kept
  dropout1 = tf.layers.dropout(
      inputs=pool, rate=0.25, training=mode == tf.estimator.ModeKeys.TRAIN)

  # Flatten tensor into a batch of vectors
  # Input Tensor Shape: [batch_size, 7, 7, 64]
  # Output Tensor Shape: [batch_size, 7 * 7 * 64]
  flat = tf.reshape(dropout1, [-1, 14 * 14 * 64])

  # Dense Layer
  # Densely connected layer with 1024 neurons
  # Input Tensor Shape: [batch_size, 14 * 14 * 64]
  # Output Tensor Shape: [batch_size, 128]
  dense = tf.layers.dense(inputs=flat, units=128, activation=tf.nn.relu)

  # Add dropout operation; 0.6 probability that element will be kept
  dropout2 = tf.layers.dropout(
      inputs=dense, rate=0.25, training=mode == tf.estimator.ModeKeys.TRAIN)

  # Logits layer
  # Input Tensor Shape: [batch_size, 1024]
  # Output Tensor Shape: [batch_size, 10]
  logits = tf.layers.dense(inputs=dropout2, units=10)

  predictions = {
      # Generate predictions (for PREDICT and EVAL mode)
      "classes": tf.argmax(input=logits, axis=1),
      # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
      # `logging_hook`.
      "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
  }
  if mode == tf.estimator.ModeKeys.PREDICT:
    return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

  # Calculate Loss (for both TRAIN and EVAL modes)
  loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

  # Configure the Training Op (for TRAIN mode)
  if mode == tf.estimator.ModeKeys.TRAIN:
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
    train_op = optimizer.minimize(
        loss=loss,
        global_step=tf.train.get_global_step())
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

  # Add evaluation metrics (for EVAL mode)
  eval_metric_ops = {
      "accuracy": tf.metrics.accuracy(
          labels=labels, predictions=predictions["classes"])}
  return tf.estimator.EstimatorSpec(
      mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)


In [ ]:


def main(unused_argv):
  # Load training and eval data
  mnist = tf.contrib.learn.datasets.load_dataset("mnist")
  train_data = mnist.train.images  # Returns np.array
  train_labels = np.asarray(mnist.train.labels, dtype=np.int32)
  eval_data = mnist.test.images  # Returns np.array
  eval_labels = np.asarray(mnist.test.labels, dtype=np.int32)

  # Create the Estimator
  mnist_classifier = tf.estimator.Estimator(
      model_fn=cnn_model_fn, model_dir="/tmp/mnist_convnet_model")

  # Set up logging for predictions
  # Log the values in the "Softmax" tensor with label "probabilities"
  tensors_to_log = {"probabilities": "softmax_tensor"}
  logging_hook = tf.train.LoggingTensorHook(
      tensors=tensors_to_log, every_n_iter=50)

  # Train the model
  train_input_fn = tf.estimator.inputs.numpy_input_fn(
      x={"x": train_data},
      y=train_labels,
      batch_size=100,
      num_epochs=None,
      shuffle=True)
  mnist_classifier.train(
      input_fn=train_input_fn,
      steps=20000,
      hooks=[logging_hook])

  # Evaluate the model and print results
  eval_input_fn = tf.estimator.inputs.numpy_input_fn(
      x={"x": eval_data},
      y=eval_labels,
      num_epochs=1,
      shuffle=False)
  eval_results = mnist_classifier.evaluate(input_fn=eval_input_fn)
  print(eval_results)


if __name__ == "__main__":
  tf.app.run()

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting MNIST-data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting MNIST-data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST-data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST-data/t10k-labels-idx1-ubyte.gz
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1a1d817850>, '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_num_ps_replicas': 0, '_tf_random_seed': None, '_master': '', '_num_worker_replicas': 1, '_task_id': 0, '_log_step_count_steps': 100, '_model_dir': '/tmp/mnist_convnet_

INFO:tensorflow:loss = 2.28609, step = 2
INFO:tensorflow:probabilities = [[ 0.11068202  0.09061854  0.09621365  0.10192529  0.10082338  0.10404242
   0.0880582   0.10902494  0.09168042  0.10693106]
 [ 0.0982655   0.09923679  0.09953755  0.09177018  0.10400628  0.10065477
   0.09352348  0.09933346  0.09898291  0.11468912]
 [ 0.10216863  0.08609206  0.10285016  0.10128414  0.10238156  0.1049917
   0.09968662  0.09517839  0.09921489  0.1061518 ]
 [ 0.10250501  0.08983664  0.10220237  0.11970532  0.09735611  0.10192399
   0.09603009  0.09760544  0.09510956  0.09772539]
 [ 0.10925043  0.08961837  0.09822313  0.10566659  0.09821638  0.11355043
   0.09140874  0.091694    0.09990523  0.10246667]
 [ 0.11686495  0.08922173  0.10123452  0.09940303  0.09895501  0.10363853
   0.09931224  0.09150426  0.09794016  0.10192557]
 [ 0.09770757  0.08638247  0.09787828  0.09871964  0.11084004  0.10699005
   0.09403347  0.10873988  0.09407657  0.10463211]
 [ 0.10185046  0.09531982  0.09614722  0.10219641  0.

INFO:tensorflow:global_step/sec: 4.1586
INFO:tensorflow:probabilities = [[ 0.1163077   0.086485    0.09897939  0.10093997  0.10520054  0.1079189
   0.09846477  0.08458018  0.10133456  0.09978891]
 [ 0.10095458  0.08187582  0.10644904  0.11271179  0.11188649  0.10789046
   0.09272991  0.09081399  0.09957106  0.0951169 ]
 [ 0.10952281  0.0936138   0.10586018  0.09162714  0.10522911  0.09781788
   0.0954641   0.09492506  0.10430284  0.10163704]
 [ 0.10314019  0.07629631  0.09987467  0.11423434  0.1082663   0.10905378
   0.09623189  0.09738576  0.09280763  0.10270921]
 [ 0.11455084  0.08688925  0.09245396  0.11057123  0.10095984  0.11800801
   0.09399773  0.08082266  0.09636826  0.10537819]
 [ 0.10946061  0.09389646  0.09223299  0.11190553  0.10994916  0.09241386
   0.09969128  0.09237088  0.09204128  0.10603791]
 [ 0.10072697  0.08860112  0.10203943  0.1011958   0.09678358  0.10376125
   0.09744664  0.10299929  0.10173815  0.10470774]
 [ 0.10265644  0.090386    0.09463977  0.11492462  0.0

INFO:tensorflow:loss = 2.28326, step = 102 (24.049 sec)
INFO:tensorflow:probabilities = [[ 0.10597587  0.08498256  0.10476826  0.10528822  0.09766334  0.11269342
   0.10290871  0.08914817  0.09994026  0.09663121]
 [ 0.09957484  0.09762913  0.10249359  0.11502581  0.09908182  0.10257207
   0.09470666  0.0943078   0.09220737  0.10240089]
 [ 0.10628662  0.07868765  0.0997207   0.10334143  0.1083796   0.1080986
   0.10454622  0.09069541  0.09780983  0.10243393]
 [ 0.11188103  0.0769999   0.1146599   0.11262067  0.09798909  0.11068238
   0.09358148  0.09489962  0.09528118  0.09140474]
 [ 0.12007832  0.08602228  0.11223293  0.10677776  0.10878594  0.09279142
   0.08911593  0.09182744  0.09792862  0.09443928]
 [ 0.1006247   0.09221494  0.10090737  0.10023568  0.10852315  0.10071299
   0.10446864  0.09119098  0.09573323  0.10538836]
 [ 0.11602432  0.08613984  0.0995554   0.10141593  0.09864584  0.11477132
   0.09769804  0.09313285  0.10172772  0.09088878]
 [ 0.10879897  0.08619917  0.10234848 